In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:78% !important; }</style>"))

In [6]:
import urllib.request
from urllib import parse
import requests as rq

import pandas as pd
import numpy as np
from datetime import datetime, date

import json

import matplotlib as mpl
import matplotlib.pylab as plt

import xlsxwriter

from dfply import *

## 개요

* open API로 제공되는 json 파일을 받아서 처리후 엑셀 데이터로 가공하여 저장

## 데이터

* 주소: https://app.swaggerhub.com/apis-docs/Promptech/public-mask-info/20200307-oas3

## 데이터 가져오기

In [7]:
# address = parse.quote('대전광역시 유성구')
address = parse.quote('서울특별시 강남구')

request_url = "https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1/storesByAddr/json?address=" + address

df = pd.DataFrame(json.loads(rq.get(request_url).text)['stores'])

## 가공처리

### 데이터 확인

In [8]:
df.shape

(461, 9)

In [9]:
df.head(10)

,addr,code,created_at,lat,lng,name,remain_stat,stock_at,type
0,"서울특별시 강남구 영동대로112길 15 103호 (삼성동, 풍림아파트상가)",11800011,2020/05/28 23:55:00,37.515519,127.061515,부활약국,some,2020/05/27 13:24:00,01
1,서울특별시 강남구 봉은사로 465 (삼성동),11800038,2020/05/28 23:55:00,37.512485,127.050491,신삼성약국,plenty,2020/05/28 09:28:00,01
2,서울특별시 강남구 봉은사로114길 42 (삼성동),11800071,2020/05/28 23:55:00,37.512172,127.065342,강남소화약국,some,2020/05/28 09:45:00,01
3,"서울특별시 강남구 삼성로 549 (삼성동, 정화빌딩)",11800178,2020/05/28 23:55:00,37.510603,127.054292,은약국,empty,2020/05/28 10:46:00,01
4,"서울특별시 강남구 논현로 88 3층 (개포동, 노엘빌딩)",11800283,2020/05/28 23:55:00,37.477822,127.045161,다나약국,break,2020/05/28 09:09:00,01
5,서울특별시 강남구 도산대로 113 1층 (신사동),11800429,2020/05/28 23:55:00,37.517022,127.020867,가나안약국,empty,2020/05/28 09:02:00,01
6,서울특별시 강남구 영동대로86길 10 1층 (대치동),11800461,2020/05/28 23:55:00,37.507824,127.064928,메디칼희민약국,empty,2020/05/28 15:52:00,01
7,서울특별시 강남구 언주로125길 6 (논현동),11800577,2020/05/28 23:55:00,37.513227,127.034779,민생약국,plenty,2020/05/28 08:15:00,01
8,서울특별시 강남구 도산대로38길 11 102-4호 (논현동),11800615,2020/05/28 23:55:00,37.519992,127.032359,인터팜이화약국,break,2020/05/28 07:40:00,01
9,서울특별시 강남구 선릉로 755 1층 (논현동),11800682,2020/05/28 23:55:00,37.522027,127.039195,인선약국,plenty,2020/05/28 17:14:00,01


* 필요한 자료 추출 및 확인

### dfply packge 사용

* 참조: https://github.com/kieferk/dfply
* 변수 생성을 추가하지 않고, 분석자의 직관을 따라 결과 산출이 가능

In [21]:
(
    df
    >> mask(X.stock_at > '2020/05/28 12:00')
    >> mask(X.remain_stat == "break")
#     >> arrange(X.stock_at, ascending = False)
#     >> select(X.stock_at)
    >> head(5)
)

,addr,code,created_at,lat,lng,name,remain_stat,stock_at,type
14,서울특별시 강남구 도산대로 522 (청담동),11801034,2020/05/28 23:55:00,37.524290,127.050155,새생명약국,break,2020/05/28 15:54:00,01
42,"서울특별시 강남구 개포로 615 101호 (개포동, 석탑프라자)",11804050,2020/05/28 23:55:00,37.492664,127.075601,솔약국,break,2020/05/28 20:02:00,01
54,서울특별시 강남구 언주로 707 2층 일부호 (논현동),11824425,2020/05/28 23:55:00,37.516223,127.035011,행복한미래약국,break,2020/05/28 15:05:00,01
74,서울특별시 강남구 압구정로36길 12 1층 103호 (신사동),11847051,2020/05/28 23:55:00,37.527642,127.031338,세진약국,break,2020/05/28 16:44:00,01
80,서울특별시 강남구 역삼로 310 2층 25호 (역삼동),11854529,2020/05/28 23:55:00,37.499476,127.048333,한솔인유로온누리약국,break,2020/05/28 16:04:00,01


* 5월 28일 당일 기준으로 plenty, some, break status 비율은 얼마나 될까?

In [59]:
(
    df
    >> mask(X.stock_at > '2020/05/28 00:00:00')
    >> group_by(X.remain_stat)
    >> summarise(CNT = n(X.code))
#     >> mutate(test = X.CNT*2)
    >> ungroup()
    >> arrange(X.CNT, ascending = False)
#     >> mutate(RATIO = X.CNT/cumsum(X.CNT))
#     >> head(10)
)

,remain_stat,CNT
3,plenty,159
0,break,104
1,empty,82
4,some,22
2,few,7


* '역상'동에 있는 약국으로 재고 상태가 'plenty'인 것을 추출

In [90]:
df2 = (
    df
    >> mask(X.addr.str.contains("역삼"))
    >> mask(X.stock_at > '2020-05-28 22:00:00')
    >> mask(X.remain_stat == "plenty")
#     >> separge(X.CNT, ascending = False)
#     >> head(5)
) 

In [87]:
df2

,addr,code,created_at,lat,lng,name,remain_stat,stock_at,type
15,서울특별시 강남구 언주로108길 15 (역삼동),11801051,2020/05/28 23:55:00,37.508541,127.040520,호원약국,plenty,2020/05/28 10:32:00,01
31,서울특별시 강남구 언주로 323 (역삼동),11802448,2020/05/28 23:55:00,37.496740,127.045112,유진약국,plenty,2020/05/28 21:07:00,01
43,"서울특별시 강남구 강남대로 416 1층 (역삼동, 창림빌딩)",11804807,2020/05/28 23:55:00,37.499813,127.027178,씨티약국,plenty,2020/05/28 09:01:00,01
47,서울특별시 강남구 강남대로 364 미왕빌딩 2층 (역삼동),11810297,2020/05/28 23:55:00,37.495550,127.029294,아름약국,plenty,2020/05/28 11:16:00,01
49,서울특별시 강남구 역삼로 243 1층 (역삼동),11815701,2020/05/28 23:55:00,37.497866,127.042833,아로마약국,plenty,2020/05/28 08:37:00,01
65,"서울특별시 강남구 역삼로 247 102호 (역삼동, 홍진빌딩)",11836539,2020/05/28 23:55:00,37.497965,127.043167,밝은약국,plenty,2020/05/28 10:26:00,01
73,"서울특별시 강남구 언주로 508 (역삼동, 상록회관내)",11846046,2020/05/28 23:55:00,37.503965,127.042768,상록약국,plenty,2020/05/28 08:38:00,01
118,서울특별시 강남구 도곡로 213 지하1층 (역삼동),11889314,2020/05/28 23:55:00,37.493083,127.042121,서울세연약국,plenty,2020/05/28 09:41:00,01
129,서울특별시 강남구 테헤란로28길 5 1층 일부호 (역삼동),11897155,2020/05/28 23:55:00,37.500772,127.038846,샬롬온누리약국,plenty,2020/05/28 09:04:00,01
145,"서울특별시 강남구 선릉로63길 9 2층 일부호 (역삼동, 백림빌딩)",12806765,2020/05/28 23:55:00,37.497171,127.051456,스카이약국,plenty,2020/05/28 11:13:00,01


## 엑셀 저장